# Import packages

In [1]:
import pandas as pd
import requests
import re
import sys
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
# Disable python warnings
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

# Functions

In [3]:
def parse_table(url):
    driver = webdriver.Chrome(executable_path='./chromedriver')
    driver.get(url)

    soup = BeautifulSoup(driver.page_source, features='html.parser')
    driver.quit()
    
    # Obtain information from class
    table = soup.find('div', {'class':'table-container'})
    
    return table

# Scrape data

In [4]:
NC = [1,2,3,4,5,6,7,8]

main_link = ["https://unfccc.int/process/transparency-and-reporting/reporting-and-review-under-the-convention/national-communications-and-biennial-reports-annex-i-parties/national-communication-submissions/first-second-third-national-communications-annex-i", "https://unfccc.int/process/transparency-and-reporting/reporting-and-review-under-the-convention/national-communications-and-biennial-reports-annex-i-parties/national-communication-submissions/first-second-third-national-communications-annex-i", "https://unfccc.int/process/transparency-and-reporting/reporting-and-review-under-the-convention/national-communications-and-biennial-reports-annex-i-parties/national-communication-submissions/first-second-third-national-communications-annex-i", "https://unfccc.int/process/transparency-and-reporting/reporting-and-review-under-the-convention/national-communications-and-biennial-reports-annex-i-parties/national-communication-submissions/fourth-national-communications-and-reports-demonstrating-progress-under-the-kyoto", "https://unfccc.int/process/transparency-and-reporting/reporting-and-review-under-the-convention/national-communications-and-biennial-reports-annex-i-parties/national-communication-submissions/fifth-national-communications-annex-i", "https://unfccc.int/process/transparency-and-reporting/reporting-and-review-under-the-convention/national-communications-and-biennial-reports-annex-i-parties/submitted-national-communications-from-annex-i-parties/sixth-national-communications-annex-i", "https://unfccc.int/NC7", "https://unfccc.int/NC8"]

zipped = list(zip(NC, main_link))

main_links = pd.DataFrame(zipped, columns=['NC', 'main_link'])

main_links

,NC,main_link
0,1,https://unfccc.int/process/transparency-and-re...
1,2,https://unfccc.int/process/transparency-and-re...
2,3,https://unfccc.int/process/transparency-and-re...
3,4,https://unfccc.int/process/transparency-and-re...
4,5,https://unfccc.int/process/transparency-and-re...
5,6,https://unfccc.int/process/transparency-and-re...
6,7,https://unfccc.int/NC7
7,8,https://unfccc.int/NC8


## NC1, NC2, NC3 links

In [6]:
url = main_links.iloc[0]['main_link']

table = parse_table(url)

# Defining dataframe
df = pd.DataFrame(columns=['country', 'NC1', 'NC2', 'NC3', 'NC4', 'NC5', 'NC6', 'NC7'])

# Collecting Data
for row in table.tbody.find_all('tr'):
    # Find all data for each column
    columns = row.find_all('td')
    
    if(columns != None):
        countries = columns[0].text.strip()
        nc1 = columns[1].find('a', href= True)
        nc2 = columns[2].find('a', href= True)
        nc3 = columns[3].find('a', href= True)
        if nc1 == None:
            nc1_link = ''
        else:
            nc1_link = nc1['href']
        if(nc2 == None):
            nc2_link = ''
        else:
            nc2_link = nc2['href']
        if(nc3 == None):
            nc3_link = ''
        else:
            nc3_link = nc3['href']
            
        df = df.append({'country': countries, 'NC1': nc1_link, 'NC2': nc2_link, 'NC3': nc3_link}, ignore_index=True)

## NC4, NC5, NC6 links

In [9]:
for i,link in enumerate(main_links[3:6]['main_link']):
    table = parse_table(link)
        
    # Define list
    nc_list = []
        
    # Collect Data
    for row in table.tbody.find_all('tr'):
        # Find all data for each column
        columns = row.find_all('p') 
    
        if(columns != None):
            countries = columns[0].text.strip()
            if not countries == 'Cyprus':  # delete Cyprus to make equal length of rows (44) - not necessary in our case
                ncs = columns[1].find('a', href= True)
                
                if(ncs == None):
                    nc_link = ''
                else:
                    nc_link = ncs['href']
                nc_list.append(nc_link)
                
    df['NC'+ str(i+4)] = nc_list

## NC7 links

In [10]:
table = parse_table(main_links.iloc[6]['main_link'])
    
# Define list
nc_list = []
    
# Collect Data
for row in table.tbody.find_all('tr'):
    # Find all data for each column
    columns = row.find_all('td') 
    
    if(columns != None):
        countries = columns[0].text.strip()
        if not countries == 'Cyprus':  # delete Cyprus to make equal length of rows (44) - not necessary for our country sample
            ncs = columns[2].find('a', href= True)
        
            if(ncs == None):
                nc_link = ''
            else:
                nc_link = ncs['href']
                
            nc_list.append(nc_link)
            
df['NC7'] = nc_list

In [11]:
df

,country,NC1,NC2,NC3,NC4,NC5,NC6,NC7
0,Annex I countries,,,,,,,
1,Australia,https://unfccc.int/documents/198646,https://unfccc.int/documents/198647,https://unfccc.int/documents/198648,https://unfccc.int/documents/198545,https://unfccc.int/documents/198435,https://unfccc.int/documents/198294,https://unfccc.int/documents/69238
2,Austria,https://unfccc.int/documents/198649,https://unfccc.int/documents/198650,https://unfccc.int/documents/198651,https://unfccc.int/documents/198546,https://unfccc.int/documents/198436,https://unfccc.int/documents/198295,https://unfccc.int/documents/64720
3,Belarus,https://unfccc.int/documents/198652,https://unfccc.int/documents/198548,https://unfccc.int/documents/198548,https://unfccc.int/documents/198548,https://unfccc.int/documents/198437,https://unfccc.int/documents/198303,/documents/199865
4,Belgium,https://unfccc.int/documents/198653,https://unfccc.int/documents/198654,https://unfccc.int/documents/198655,https://unfccc.int/documents/198551,https://unfccc.int/documents/198441,https://unfccc.int/documents/198304,https://unfccc.int/documents/198234
5,Bulgaria,https://unfccc.int/documents/198656,https://unfccc.int/documents/198657,https://unfccc.int/documents/198658,https://unfccc.int/documents/198553,https://unfccc.int/documents/198442,https://unfccc.int/documents/198305,https://unfccc.int/documents/198224
6,Canada,https://unfccc.int/documents/198659,https://unfccc.int/documents/198660,https://unfccc.int/documents/198661,https://unfccc.int/documents/198554,https://unfccc.int/documents/198444,https://unfccc.int/documents/198320,https://unfccc.int/documents/198352
7,Croatia,https://unfccc.int/documents/198662,https://unfccc.int/documents/198575,https://unfccc.int/documents/198575,https://unfccc.int/documents/198575,https://unfccc.int/documents/198445,https://unfccc.int/documents/198322,https://unfccc.int/documents/66923
8,Czech Republic,,https://unfccc.int/documents/198663,https://unfccc.int/documents/198666,https://unfccc.int/documents/198578,https://unfccc.int/documents/198446,https://unfccc.int/documents/198334,https://unfccc.int/documents/198236
9,Denmark,https://unfccc.int/documents/198673,https://unfccc.int/documents/198674,https://unfccc.int/documents/198675,https://unfccc.int/documents/198579,https://unfccc.int/documents/198448,https://unfccc.int/documents/198338,https://unfccc.int/documents/28946


# Short data cleaning & save

In [12]:
df = df[1:] # remove first row

In [13]:
wsite_address = 'https://unfccc.int'
df.at[3,'NC7'] = wsite_address + '/documents/624772'  # correct values for Belarus 
df.at[43,'NC7'] = wsite_address + '/documents/624756' # correct values for the USA

In [15]:
df.to_csv('nc_links.csv')